#### **Sign to Speech Converter (Dataset Generation)**
 This Notebook shows how the dataset is created for our Sign to Speech converter.
 
 Also the paths for creating folders correspond to the local machine on which this notebook was tested.
 
 
 **Please ensure to make necessary changes to the paths before executing the cells** 

#### Import all the necessary libraries

In [ ]:
import cv2
import os   # accessing folder paths

#### Create dictionary classes for different data generators

We are creating dictionary objects using which we create folders for our dataset

These same dictionaries are used  later during prediction

In [ ]:
num_classes = {1:'1',2:'2',3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8',
           9:'9'}
alpha_classes = {1:'A',2:'B',3:'C',4:'D',5:'E',6:'F',7:'G',8:'H',9:'I',
           10:'J',11:'K',12:'L',13:'M',14:'N',15:'O',16:'P',17:'Q',18:'R',19:'S',20:'T',21:'U',
           22:'V',23:'W',24:'X',25:'Y',26:'Z'}
mylist = [x for x in alpha_classes.values()]
print(mylist[8:])
words_data = {1:'All_The_Best', 2:'Hi!!', 3: 'I_Love_you', 4: 'No', 5:'Super!!', 6:'Yes'}

#### Creating Directories for datasets
Next we create directories for our datasets. First create directories in the name of class keys.

Make sure to **change the path in mkdir** to path where you want to store dataset in your machine

We can store entire dataset at same path, However when creating the dataset we have to show the entire
gestures for our dataset in single run. So we create and store datasets for each class separately.

**Note**: You need to run this only once.

In [ ]:
# Creates directories for alphabets dataset
# Change the path in mkdir to path where you want to store the dataset
print('Creating Folders for alphabets data. Please wait...')
for dir_name in alpha_classes.values():
    !mkdir {'G:\\gestures\\alpha_data\\train\\' + dir_name}
    !mkdir {'G:\\gestures\\alpha_data\\test\\' + dir_name}
print('Done.')

In [ ]:
# Creates directories for numbers
# Change the path in mkdir to path where you want to store the dataset
print('Creating Folders for numbers data. Please wait...')
for dir_name in num_classes.values():
    !mkdir {'G:\\gestures\\numbers_data\\train\\' + dir_name}
    !mkdir {'G:\\gestures\\numbers_data\\test\\' + dir_name}
print('Done.')

In [ ]:
# creates directories for words
# RUN THIS ONLY ONCE 
# Change the path in mkdir to path where you want to store the dataset
print('Creating Folders for words data. Please wait....')
for dir_name in words_data.values():
    !mkdir {'G:\\gestures\\words_data\\test\\' + dir_name}
    !mkdir {'G:\\gestures\\words_data\\train\\' + dir_name}
    
print('Done!!!')

In [ ]:
background = None
accumulated_weight = 0.7
mask_color = (0.0,0.0,0.0)

ROI_top = 100
ROI_bottom = 300
ROI_right = 300
ROI_left = 500

#### Helper functions


In [ ]:
# This function is used to calculate accumulated_weights in the frame
def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [ ]:
# This function segments the hand region found in the frame, if not found returns None.
def segment_hand(frame, threshold=50):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    
    _ , thresholded = cv2.threshold(diff, threshold, 255,cv2.THRESH_BINARY)
    
    edges = cv2.Canny(thresholded, threshold1= 50, threshold2=250)
    cv2.imshow('edges',thresholded)
    
     #Fetching contours in the frame (These contours can be of hand
    #or any other object in foreground) …

    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # If length of contours list = 0, means we didn't get any
    #contours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand
        # contour_info = [(c, cv2.contourArea(c),) for c in contours[1]]

        #cntrs, heirs = cv2.findContours(thresholded.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

        contour_info = [(c, cv2.contourArea(c),) for c in contours]
        #for c in contours[1]:
        #    contour_info.append((c,cv2.contourArea(c),))
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        # Returning the hand segment(max contour) and the
  # thresholded image of hand and contour_info list
    return (thresholded, hand_segment_max_cont, contour_info)

#### Creating the dataset for each Dictionary
 **Note** : Run this cell only when you want to create data

 Make sure you have created corresponding directories. By running all the above previous cells 

In [ ]:
## Initialize the base dir, train_dir, test_dir
#base_dir = 'G:\\gestures\\words_data\\' # un comment this to create words dataset
base_dir = 'G:\\gestures\\alpha_data\\' # uncomment this to create alphabet dataset
#base_dir = 'G:\\gestures\\numbers_data\\'  # uncomment this to create numbers dataset

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir,'test')

#### You need to show gestures for each of the elements you are creating dataset for

See ISL one hand.gif and ISL numbers.jpg in assets folder to know about the signs.

In [ ]:
import time
no_of_test_imgs_to_generate = 70
no_of_train_imgs_to_generate = 700
# change alpha_classes in for loop to dictionary name for which you want to create dataset for.
# You need to show the gesture for each of the elements in dictionary.
# You can alter the number of images to be generated in above lines.

for element in alpha_classes.values(): 
    cam = cv2.VideoCapture(0)
    num_frames = 0
    num_imgs_taken = 0
    time.sleep(7)
    print('#################################################')
    print(f'Show sign for {element}!')

    print(f'Creating data for {element}.....')
    
    while True:
        ret, frame = cam.read()

        # flipping the frame to prevent inverted image of captured frame...
        frame = cv2.flip(frame, 1)

        frame_copy = frame.copy()

        roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

        gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)

        # For initial 60 frames we fetch the background.
        if num_frames < 60:
            cal_accum_avg(gray_frame, accumulated_weight)
            if num_frames <= 59:
                cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT",
                            (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
                
        #Time to configure the hand specifically into the ROI...
        elif num_frames <= 300: 

            hand = segment_hand(gray_frame)
            cv2.putText(frame_copy, "Adjust hand gesture for..",
                            (200, 400), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,0,255), 2)
                
            if hand is not None:
                
                thresholded, hand_segment, contour_info = hand

                # Draw contours around hand segment
                cv2.drawContours(frame_copy, [hand_segment + (ROI_right,
                ROI_top)], -1, (255, 0, 0),1)
                
                cv2.putText(frame_copy, str(num_frames)+"For" + str(element),
                            (70, 45), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,0,255), 2)

                # Also display the thresholded image
                cv2.imshow("Thresholded Hand Image", thresholded)
        
        else: 
            
            # Segmenting the hand region...
            hand = segment_hand(gray_frame)
            
            # if we are able to detect the hand then proceed to create dataset
            if hand is not None:
                
                # unpack the thresholded img and the max_contour...
                thresholded, hand_segment,contour_info = hand

                # Drawing contours around hand segment
                cv2.drawContours(frame_copy, [hand_segment + (ROI_right,
                ROI_top)], -1, (255, 0, 0),1)
                
                cv2.putText(frame_copy, str(num_frames), (70, 45),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                cv2.putText(frame_copy,"Adjust hand gesture for..",(200, 400),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1,(0,0,                              255)                                                                                                                   , 2)
                # Displaying the thresholded image
                cv2.imshow("Thresholded Hand Image", thresholded)
                
                ################################################################ 
                # Make below changes while generating data for test directory
                # no_of_train_imgs_to_generate -> no_of_test_imgs_to_generate
                # train_dir -> test_dir
                ################################################################
                if num_imgs_taken <= no_of_train_imgs_to_generate:
                    cv2.imwrite(train_dir + '\\' +str(element)+"\\" + str(num_imgs_taken) + '.jpg',                                   thresholded)
                else:
                    break
                num_imgs_taken +=1
                
            else:
                cv2.putText(frame_copy, 'No hand detected...', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)



        # Drawing ROI on frame copy
        cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,ROI_bottom), (255,128,0), 3)
        
        cv2.putText(frame_copy, "Sign languge recognition_ _ _", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
        
        # increment the number of frames for tracking
        num_frames += 1

        # Display the frame with segmented hand
        cv2.imshow("Sign Detection", frame_copy)

        # Closing windows with Esc key...(any other key with ord can be used too.)
        k = cv2.waitKey(1) & 0xFF

        if k == 27:
            break

    # Releasing the camera & destroying all the windows...

    cv2.destroyAllWindows()
    cam.release()
    
    print('Done!')

In [ ]:
cam.release()